https://classic.curve.fi/files/stableswap-paper.pdf

* Uniswap with Leverage
* Fiat Savings Account for Liquidity Providers
* 100X lower slippage than uniswap on stablecoins

In [ ]:
int(0.004 * 10**10)

40000000

In [ ]:
int(0.5 * 10**10)

5000000000

In [ ]:
from gmpy2 import mpz
import param as pm

In [ ]:
class CurvePool(pm.Parameterized):
    A = pm.Integer(doc="Amplification Coefficient.")
    _D = pm.Integer(doc="Virtual total balance or pool coin balances in native token units.")
    n = pm.Integer(doc="Number of coins.")
    rates = pm.List(item_type=int, doc="Precision and rate adjustments.")
    tokens = pm.Integer(doc="LP token supply.")
    fee = pm.Integer(40000000, doc="Fee with 10**10 precision.")
    fee_mul = pm.Integer(doc="Fee multiplier for dynamic fee pools.")
    admin_fee = pm.Integer(5000000000, doc="Percentage of fee with 10**10 precision.")
    balances = pm.List(item_type=int)
    
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.balances = [self._D // n * 10**18 // _p for _p in self.rates]
        self.tokens = self.tokens or self.D()
        self.r = False
        self.n_total = self.n
        self.admin_balances = [0] * self.n
        
        
    def _xp(self):
        return self._xp_mem(self.rates, self.balances)
    
    def _xp_mem(self, rates, balances):
        return [x * p // 10**18 for x, p in zip(balances, rates)]
    
    def D(self, xp=None):
        xp = xp or self._xp()
        return self.get_D(xp, self.A)
    
    def get_D(self, xp, A):
        Dprev = 0
        S = sum(xp)
        D = S
        n = self.n
        Ann = A * n
        D = mpz(D)
        Ann = mpz(Ann)
        while abs(D - Dprev) > 1:
            D_P = D
            for x in xp:
                D_P = D_P * D // (n * x)
            Dprev = D
            D = (Ann * S + D_P * n) * D // ((Ann - 1) * D + (n + 1) * D_P)

        D = int(D)
        return D
    
    def get_D_mem(self, balances, A):
        xp = [x * p // 10**18 for x, p in zip(balances, self.rates)]
        return self.get_D(xp, A)
    
    def get_y(self, i, j, x, xp):
        xx = xp[:]
        D = self.D(xx)
        D = mpz(D)
        xx[i] = x  # x is quantity of underlying asset brought to 1e18 precision
        n = self.n
        xx = [xx[k] for k in range(n) if k != j]
        Ann = self.A * n
        c = D
        for y in xx:
            c = c * D // (y * n)
        c = c * D // (n * Ann)
        b = sum(xx) + D // Ann - D
        y_prev = 0
        y = D
        while abs(y - y_prev) > 1:
            y_prev = y
            y = (y**2 + c) // (2 * y + b)
        y = int(y)
        return y
    
    def get_y_D(self, A, i, xp, D):
        D = mpz(D)
        n = self.n
        xx = [xp[k] for k in range(n) if k != i]
        S = sum(xx)
        Ann = A * n
        c = D
        for y in xx:
            c = c * D // (y * n)
        c = c * D // (n * Ann)
        b = S + D // Ann
        y_prev = 0
        y = D
        while abs(y - y_prev) > 1:
            y_prev = y
            y = (y**2 + c) // (2 * y + b - D)
        y = int(y)
        return y
    
    def exchange(self, i, j, dx):
        xp = self._xp()
        x = xp[i] + dx * self.rates[i] // 10**18
        y = self.get_y(i, j, x, xp)
        dy = xp[j] - y - 1

        if self.fee_mul is None:
            fee = dy * self.fee // 10**10
        else:
            fee = dy * self.dynamic_fee((xp[i] + x) // 2, (xp[j] + y) // 2) // 10**10

        admin_fee = fee * self.admin_fee // 10**10

        # Convert all to real units
        rate = self.rates[j]
        dy = (dy - fee) * 10**18 // rate
        fee = fee * 10**18 // rate
        admin_fee = admin_fee * 10**18 // rate
        assert dy >= 0

        self.balances[i] += dx
        self.balances[j] -= dy + admin_fee
        self.admin_balances[j] += admin_fee
        return dy, fee
    
    def calc_withdraw_one_coin(self, token_amount, i, use_fee=True):
        A = self.A
        xp = self._xp()
        D0 = self.D()
        D1 = D0 - token_amount * D0 // self.tokens

        new_y = self.get_y_D(A, i, xp, D1)
        dy_before_fee = (xp[i] - new_y) * 10**18 // self.rates[i]

        xp_reduced = xp
        if self.fee and use_fee:
            n_coins = self.n
            _fee = self.fee * n_coins // (4 * (n_coins - 1))

            for j in range(n_coins):
                dx_expected = 0
                if j == i:
                    dx_expected = xp[j] * D1 // D0 - new_y
                else:
                    dx_expected = xp[j] - xp[j] * D1 // D0
                xp_reduced[j] -= _fee * dx_expected // 10**10

        dy = xp[i] - self.get_y_D(A, i, xp_reduced, D1)
        dy = (dy - 1) * 10**18 // self.rates[i]

        if use_fee:
            dy_fee = dy_before_fee - dy
            return dy, dy_fee

        return dy
    
    def add_liquidity(self, amounts):
        mint_amount, fees = self.calc_token_amount(amounts, use_fee=True)
        self.tokens += mint_amount

        balances = self.balances
        afee = self.admin_fee
        admin_fees = [f * afee // 10**10 for f in fees]
        new_balances = [
            bal + amt - fee for bal, amt, fee in zip(balances, amounts, admin_fees)
        ]
        self.balances = new_balances
        self.admin_balances = [t + a for t, a in zip(self.admin_balances, admin_fees)]

        return mint_amount
    
    def remove_liquidity_one_coin(self, token_amount, i):
        dy, dy_fee = self.calc_withdraw_one_coin(token_amount, i, use_fee=True)
        admin_fee = dy_fee * self.admin_fee // 10**10
        self.balances[i] -= dy + admin_fee
        self.admin_balances[i] += admin_fee
        self.tokens -= token_amount
        return dy, dy_fee
    
    def calc_token_amount(self, amounts, use_fee=False):
        A = self.A
        old_balances = self.balances
        D0 = self.get_D_mem(old_balances, A)

        new_balances = self.balances[:]
        for i in range(self.n):
            new_balances[i] += amounts[i]
        D1 = self.get_D_mem(new_balances, A)

        mint_balances = new_balances[:]

        if use_fee:
            _fee = self.fee * self.n // (4 * (self.n - 1))

            fees = [0] * self.n
            for i in range(self.n):
                ideal_balance = D1 * old_balances[i] // D0
                difference = abs(ideal_balance - new_balances[i])
                fees[i] = _fee * difference // 10**10
                mint_balances[i] -= fees[i]

        D2 = self.get_D_mem(mint_balances, A)

        mint_amount = self.tokens * (D2 - D0) // D0

        if use_fee:
            return mint_amount, fees

        return mint_amount
    
    def get_virtual_price(self):
        return self.D() * 10**18 // self.tokens
    
    def dynamic_fee(self, xpi, xpj):
        xps2 = xpi + xpj
        xps2 *= xps2  # Doing just ** 2 can overflow apparently
        return (self.fee_mul * self.fee) // (
            (self.fee_mul - 10**10) * 4 * xpi * xpj // xps2 + 10**10
        )
    
    def dydxfee(self, i, j):
        return self.dydx(i, j, use_fee=True)
    
    def dydx(self, i, j, use_fee=False):
        xp = self._xp()
        return self._dydx(i, j, xp, use_fee)
    
    def _dydx(self, i, j, xp, use_fee):
        xi = xp[i]
        xj = xp[j]
        n = self.n
        A = self.A
        D = self.D(xp)
        D_pow = mpz(D) ** (n + 1)
        x_prod = prod(xp)
        A_pow = A * n ** (n + 1)
        dydx = (xj * (xi * A_pow * x_prod + D_pow)) / (
            xi * (xj * A_pow * x_prod + D_pow)
        )

        if use_fee:
            if self.fee_mul is None:
                fee_factor = self.fee / 10**10
            else:
                fee_factor = self.dynamic_fee(xi, xj) / 10**10
        else:
            fee_factor = 0

        dydx *= 1 - fee_factor

        return float(dydx)

In [ ]:
cp = CurvePool()

In [ ]:
cp.D()

0

In [ ]:
mpz?

Init signature: mpz(self, /, *args, **kwargs)
Docstring:     
mpz() -> mpz(0)

     If no argument is given, return mpz(0).

mpz(n) -> mpz

     Return an 'mpz' object with a numeric value 'n' (truncating n
     to its integer part if it's a Fraction, 'mpq', float or 'mpfr').

mpz(s[, base=0]):

     Return an 'mpz' object from a string 's' made of digits in the
     given base.  If base=0, binary, octal, or hex Python strings
     are recognized by leading 0b, 0o, or 0x characters, otherwise
     the string is assumed to be decimal. Values for base can range
     between 2 and 62.
Type:           type
Subclasses:     

In [ ]:
pm.List?

Init signature:
pm.List(
    default=[],
    class_=None,
    item_type=None,
    instantiate=True,
    bounds=(0, None),
    **params,
)
Docstring:     
Parameter whose value is a list of objects, usually of a specified type.

The bounds allow a minimum and/or maximum length of
list to be enforced.  If the item_type is non-None, all
items in the list are checked to be of that type.

`class_` is accepted as an alias for `item_type`, but is
deprecated due to conflict with how the `class_` slot is
used in Selector classes.
Init docstring:
Initialize a new Parameter object and store the supplied attributes:

default: the owning class's value for the attribute represented
by this Parameter, which can be overridden in an instance.

doc: docstring explaining what this parameter represents.

label: optional text label to be used when this Parameter is
shown in a listing. If no label is supplied, the attribute name
for this parameter in the owning Parameterized object is used.

precedence: a n